In [1]:
!pip install gradio tensorflow pillow --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import time

# ---------------- MODEL ------------------
model = tf.keras.models.load_model(
    "/content/drive/MyDrive/ecobot/trained_models/ecobot_updated.keras",
    compile=False
)

CLASSES = ['Dry', 'E-waste', 'Manual', 'Wet']

BIN_COLOR_MAP = {
    "Dry": "#2AB3C6",
    "Wet": "#2ECC71",
    "E-waste": "#FFA533",
    "Manual": "#9E9E9E"
}

ICONS = {
    "Dry": "",
    "Wet": "",
    "E-waste": "",
    "Manual": ""
}

DISPOSAL_GUIDE = {
    "Dry": "🗑️ Dispose in BLUE bin.",
    "Wet": "🗑️ Dispose in GREEN bin.",
    "E-waste": "⚡ Use authorized E-waste center.",
    "Manual": "🧤 Needs manual sorting.",
}

# ---------------- PREDICT FUNCTION ------------------
def predict(img, progress=gr.Progress()):
    progress(0, desc="Processing...")
    time.sleep(0.2)

    img = img.resize((224,224))
    arr = np.expand_dims(np.array(img)/255.0, axis=0)

    progress(0.5, desc="Classifying...")
    preds = model.predict(arr)

    idx = np.argmax(preds)
    confidence = float(np.max(preds)*100)
    label = CLASSES[idx]

    return label, confidence, BIN_COLOR_MAP[label], ICONS[label], DISPOSAL_GUIDE[label]


# ---------------- UI ------------------
with gr.Blocks(title="♻SmartSort") as demo:

    # CLEAN DARK THEME + CUSTOM BUTTON CSS
    gr.HTML("""
    <style>

    html, body, .gradio-container, #root, #root > div:first-child {
        margin: 0 !important;
        padding: 0 !important;
    }

    .block, .padded, .wrap, .main {
        margin-top: 0 !important;
        padding-top: 0 !important;
    }

    body {
        background: linear-gradient(135deg, #0f1412, #1b2420);
        font-family: Arial;
        overflow-x: hidden;
    }

    h1 { color: #1E9E74 !important; margin-top: 20px !important; }
    p  { color: #d0fbe9 !important; }

    .about-box {
        padding: 18px;
        border-radius: 10px;
        background: rgba(30, 158, 116, 0.12);
        border-left: 6px solid #1E9E74;
        font-size: 16px;
        color: #d9ffec;
        line-height: 1.6;
        box-shadow: 0 0 12px rgba(0,0,0,0.4);
    }

    .about-header {
        color: #32d1a3;
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 10px;
    }

    .about-section-title {
        color: #32d1a3;
        font-weight: bold;
        margin-top: 12px;
    }

    .predict-card {
        padding: 25px;
        border-radius: 20px;
        color: white;
        text-align: center;
        font-size: 28px;
        margin-top: 15px;
        box-shadow: 0 6px 20px rgba(0,0,0,0.35);
    }

    .tip-box {
        padding: 18px;
        border-radius: 15px;
        background: rgba(255,255,255,0.07);
        font-size: 18px;
        font-weight: 600;
        margin-top: 12px;
        color: #d7ffee;
        box-shadow: 0 0 18px rgba(0,0,0,0.3);
    }

    /* ⭐ Custom classify button ⭐ */
    .smart-btn {
        background: linear-gradient(135deg, #114f3a, #1E9E74) !important;
        color: white !important;
        font-weight: bold !important;
        padding: 14px 28px !important;
        border-radius: 12px !important;
        border: none !important;
        font-size: 18px !important;
        box-shadow: 0 4px 12px rgba(0,255,180,0.2) !important;
        transition: 0.2s ease-in-out !important;
    }

    .smart-btn:hover {
        box-shadow: 0 6px 18px rgba(0,255,200,0.35) !important;
        transform: translateY(-2px) !important;
    }

    </style>
    """)

    gr.HTML("""
    <h1 style='text-align:center; font-size:45px; font-weight:900;'>
        ♻ SmartSort - Smart Waste Sorter
    </h1>
    <p style='text-align:center; font-size:17px; margin-top:-5px;'>
        Real-time waste classification into Dry, Wet, E-waste, or Manual categories.
    </p>
    """)

    with gr.Accordion("ℹ️ About SmartSort", open=False):
        gr.HTML("""
        <div class='about-box'>
            <div class='about-header'>About SmartSort</div>

            SmartSort is a real-time waste classification system powered by
            <b>MobileNetV2</b>. It promotes sustainable waste management.

            <div class='about-section-title'>🌟 Features</div>
            <ul>
                <li>Real-time image-based classification</li>
                <li>Mobile camera support</li>
                <li>Instant disposal guidance</li>
            </ul>

            <div class='about-section-title'>♻ Waste Categories</div>
            <ul>
                <li>Dry</li>
                <li>Wet</li>
                <li>E-waste</li>
                <li>Manual</li>
            </ul>
        </div>
        """)

    with gr.Column():

        img_input = gr.Image(
            type="pil",
            sources=["upload","webcam"],
            label="Upload or Capture Waste Image",
            height=300
        )

        pred_html = gr.HTML()
        tip_html = gr.HTML()

        # ⭐ CUSTOM GREEN BUTTON ⭐
        classify_btn = gr.Button("🔍  Classify Waste", elem_classes="smart-btn")

        def classify(img):
            label, conf, color, icon, tip = predict(img)
            return (
                f"<div class='predict-card' style='background:{color};'>{label}<br><span style='font-size:18px;'>Confidence: {conf:.2f}%</span></div>",
                f"<div class='tip-box' style='border-left:6px solid {color};'>{tip}</div>"
            )

        classify_btn.click(classify, inputs=img_input, outputs=[pred_html, tip_html])

    gr.HTML("<center><p style='color:#6affc4; margin-top:30px;'>Made with ❤️ by SmartSort | Smart Waste Classification © 2025</p></center>")

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ba7ac086dba471c31.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
